In [ ]:
from langchain import OpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType
)
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler

import tiktoken
from typing import List

## Streaming Final Agent Output

If you only want the final output of an agent to be streamed, you can use the callback `FinalStreamingStdOutCallbackHandler`. For this, the underlying LLM has to support streaming as well.

In [ ]:
def get_tokens(input: str) -> List[str]:
    tokens = []
    enc_dec = tiktoken.get_encoding("p50k_base")

    for idx in enc_dec.encode(input):
        tokens.append(str(enc_dec.decode_single_token_bytes(idx))[2:-1])

    return tokens

In [ ]:
llm = OpenAI(
    streaming = True,
    callbacks = [FinalStreamingStdOutCallbackHandler()],
    temperature = 0,
    openai_api_key = open("openai_api.txt").read()
)

In [ ]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = False
)

agent.run("It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany.")

 Konrad Adenauer became Chancellor of Germany in 1949, which was 74 years ago in 2023.

'Konrad Adenauer became Chancellor of Germany in 1949, which was 74 years ago in 2023.'

By default we assume that the token sequence "Final", "Answer", ":" indicates that the agent has reached an answers. We can, however, also pass a custom sequence to use as answer prefix:
```
answer_prefix_tokens=["The", "answer", ":"]
```

For convenience, the callback automatically strips whitespaces and new line characters when comparing to answer_prefix_tokens. I.e., if answer_prefix_tokens = ["The", " answer", ":"] then both ["\nThe", " answer", ":"] and ["The", " answer", ":"] would be recognized a the answer prefix.

-

If you don't know the tokenized version of your answer prefix, you can determine it with the following code:

In [ ]:
class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs) -> None:
        # print every token on a new line
        print(f"#{token}#")

llm = OpenAI(
    streaming = True,
    callbacks = [MyCallbackHandler()],
    temperature = 0,
    openai_api_key = open("openai_api.txt").read()
)

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

agent.run("It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany.")

# I#
# need#
# to#
# find#
# out#
# when#
# Kon#
#rad#
# Aden#
#auer#
# became#
# Chancellor#
# of#
# Germany#
#.#
#
Action#
#:#
# Wikipedia#
#
Action#
# Input#
#:#
# Kon#
#rad#
# Aden#
#auer#
##
# I#
# now#
# know#
# the#
# date#
# Kon#
#rad#
# Aden#
#auer#
# became#
# Chancellor#
# of#
# Germany#
#.#
#
Final#
# Answer#
#:#
# Kon#
#rad#
# Aden#
#auer#
# became#
# Chancellor#
# of#
# Germany#
# in#
# 1949#
#,#
# which#
# was#
# 74 years#
# ago#
# in#
# 20#
#23#
#.#
##


'Konrad Adenauer became Chancellor of Germany in 1949, which was 74 years ago in 2023.'